# Language Translation
In this project, you’re going to take a peek into the realm of neural network machine translation.  You’ll be training a sequence to sequence model on a dataset of English and French sentences that can translate new sentences from English to French.
## Get the Data
Since translating the whole language of English to French will take lots of time to train, we have provided you with a small portion of the English corpus.

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

source_path = 'data/small_vocab_en'
target_path = 'data/small_vocab_fr'
source_text = helper.load_data(source_path)
target_text = helper.load_data(target_path)

## Explore the Data
Play around with view_sentence_range to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in source_text.split()})))

sentences = source_text.split('\n')
word_counts = [len(sentence.split()) for sentence in sentences]
print('Number of sentences: {}'.format(len(sentences)))
print('Average number of words in a sentence: {}'.format(np.average(word_counts)))
print(source_text[0:10].split())
print()
print('English sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(source_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))
print()
print('French sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(target_text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 227
Number of sentences: 137861
Average number of words in a sentence: 13.225277634719028
['new', 'jersey']

English sentences 0 to 10:
new jersey is sometimes quiet during autumn , and it is snowy in april .
the united states is usually chilly during july , and it is usually freezing in november .
california is usually quiet during march , and it is usually hot in june .
the united states is sometimes mild during june , and it is cold in september .
your least liked fruit is the grape , but my least liked is the apple .
his favorite fruit is the orange , but my favorite is the grape .
paris is relaxing during december , but it is usually chilly in july .
new jersey is busy during spring , and it is never hot in march .
our least liked fruit is the lemon , but my least liked is the grape .
the united states is sometimes busy during january , and it is sometimes warm in november .

French sentences 0 to 10:
new jersey est parfois calme p

## Implement Preprocessing Function
### Text to Word Ids
As you did with other RNNs, you must turn the text into a number so the computer can understand it. In the function `text_to_ids()`, you'll turn `source_text` and `target_text` from words to ids.  However, you need to add the `<EOS>` word id at the end of `target_text`.  This will help the neural network predict when the sentence should end.

You can get the `<EOS>` word id by doing:
```python
target_vocab_to_int['<EOS>']
```
You can get other word ids using `source_vocab_to_int` and `target_vocab_to_int`.

In [3]:
def text_to_ids(source_text, target_text, source_vocab_to_int, target_vocab_to_int):
    """
    Convert source and target text to proper word ids
    :param source_text: String that contains all the source text.
    :param target_text: String that contains all the target text.
    :param source_vocab_to_int: Dictionary to go from the source words to an id
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: A tuple of lists (source_id_text, target_id_text)
    """
    # TODO: Implement Function
    # split texts on new line character into seperate lists, split lists on space, lookup id from appropriate dict
    # added EOS value to target lists
    source_id_text = [[source_vocab_to_int.get(word, 2) for word in phrase.split()] for phrase in source_text.split('\n')]
    target_id_text = [[target_vocab_to_int.get(word, 2) for word in phrase.split()] + [target_vocab_to_int['<EOS>']] for phrase in target_text.split('\n')]

    return (source_id_text, target_id_text)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_text_to_ids(text_to_ids)

Tests Passed


### Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [4]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
helper.preprocess_and_save_data(source_path, target_path, text_to_ids)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np
import helper
import problem_unittests as tests

(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()

### Check the Version of TensorFlow and Access to GPU
This will check to make sure you have the correct version of TensorFlow and access to a GPU

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.1'), 'Please use TensorFlow version 1.1 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


## Build the Neural Network
You'll build the components necessary to build a Sequence-to-Sequence model by implementing the following functions below:
- `model_inputs`
- `process_decoder_input`
- `encoding_layer`
- `decoding_layer_train`
- `decoding_layer_infer`
- `decoding_layer`
- `seq2seq_model`

### Input
Implement the `model_inputs()` function to create TF Placeholders for the Neural Network. It should create the following placeholders:

- Input text placeholder named "input" using the TF Placeholder name parameter with rank 2.
- Targets placeholder with rank 2.
- Learning rate placeholder with rank 0.
- Keep probability placeholder named "keep_prob" using the TF Placeholder name parameter with rank 0.
- Target sequence length placeholder named "target_sequence_length" with rank 1
- Max target sequence length tensor named "max_target_len" getting its value from applying tf.reduce_max on the target_sequence_length placeholder. Rank 0.
- Source sequence length placeholder named "source_sequence_length" with rank 1

Return the placeholders in the following the tuple (input, targets, learning rate, keep probability, target sequence length, max target sequence length, source sequence length)

In [7]:
def model_inputs():
    """
    Create TF Placeholders for input, targets, learning rate, and lengths of source and target sequences.
    :return: Tuple (input, targets, learning rate, keep probability, target sequence length,
    max target sequence length, source sequence length)
    """
    inputs = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, None)
    keep_probability = tf.placeholder(tf.float32, None, name='keep_prob')
    target_sequence_length = tf.placeholder(tf.int32, [None], name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_sequence_length')
    source_sequence_length = tf.placeholder(tf.int32, [None], name='source_sequence_length')
    
    return (inputs, targets, learning_rate, keep_probability, target_sequence_length,
    max_target_sequence_length, source_sequence_length)



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_model_inputs(model_inputs)

Tests Passed


### Process Decoder Input
Implement `process_decoder_input` by removing the last word id from each batch in `target_data` and concat the GO ID to the begining of each batch.

In [8]:
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    """
    Preprocess target data for encoding
    :param target_data: Target Placehoder
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param batch_size: Batch Size
    :return: Preprocessed target data
    """
    # TODO: Implement Function
    end = tf.strided_slice(target_data, [0,0], [batch_size, -1], [1,1]) # remove last id
    pp_target_data = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), end], 1) # add go

    return pp_target_data

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_process_encoding_input(process_decoder_input)

Tests Passed


### Encoding
Implement `encoding_layer()` to create a Encoder RNN layer:
 * Embed the encoder input using [`tf.contrib.layers.embed_sequence`](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/embed_sequence)
 * Construct a [stacked](https://github.com/tensorflow/tensorflow/blob/6947f65a374ebf29e74bb71e36fd82760056d82c/tensorflow/docs_src/tutorials/recurrent.md#stacking-multiple-lstms) [`tf.contrib.rnn.LSTMCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell) wrapped in a [`tf.contrib.rnn.DropoutWrapper`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/DropoutWrapper)
 * Pass cell and embedded input to [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)

In [9]:
from imp import reload
reload(tests)

def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob, 
                   source_sequence_length, source_vocab_size, 
                   encoding_embedding_size):
    """
    Create encoding layer
    :param rnn_inputs: Inputs for the RNN
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param keep_prob: Dropout keep probability
    :param source_sequence_length: a list of the lengths of each sequence in the batch
    :param source_vocab_size: vocabulary size of source data
    :param encoding_embedding_size: embedding size of source data
    :return: tuple (RNN output, RNN state)
    """
    # TODO: Implement Function
    enc_input_emb = tf.contrib.layers.embed_sequence(rnn_inputs, source_vocab_size, encoding_embedding_size)
    
    def make_cell(rnn_size):
        return tf.contrib.rnn.LSTMCell(rnn_size, initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
    
    stacked_lstm = tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.MultiRNNCell(
                                                [make_cell(rnn_size) for _ in range(num_layers)]),
                                                input_keep_prob=keep_prob)
    
    rnn_output, rnn_state = tf.nn.dynamic_rnn(stacked_lstm, enc_input_emb, sequence_length=source_sequence_length, dtype=tf.float32)
    return (rnn_output, rnn_state)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_encoding_layer(encoding_layer)

Tests Passed


### Decoding - Training
Create a training decoding layer:
* Create a [`tf.contrib.seq2seq.TrainingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/TrainingHelper) 
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [10]:

def decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                         target_sequence_length, max_summary_length, 
                         output_layer, keep_prob):
    """
    Create a decoding layer for training
    :param encoder_state: Encoder State
    :param dec_cell: Decoder RNN Cell
    :param dec_embed_input: Decoder embedded input
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_summary_length: The length of the longest sequence in the batch
    :param output_layer: Function to apply the output layer
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing training logits and sample_id
    """
    # TODO: Implement Function
    train_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                     sequence_length=target_sequence_length,
                                                     time_major=False)
    
    basic_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, train_helper, encoder_state, output_layer)
    
    basic_decoder_output = tf.contrib.seq2seq.dynamic_decode(basic_decoder, 
                                                             impute_finished=True,
                                                             maximum_iterations = max_summary_length)[0]
    
    return basic_decoder_output #Basic Decoder Output object; tensor, sample_id



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_train(decoding_layer_train)

Tests Passed


### Decoding - Inference
Create inference decoder:
* Create a [`tf.contrib.seq2seq.GreedyEmbeddingHelper`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/GreedyEmbeddingHelper)
* Create a [`tf.contrib.seq2seq.BasicDecoder`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/BasicDecoder)
* Obtain the decoder outputs from [`tf.contrib.seq2seq.dynamic_decode`](https://www.tensorflow.org/api_docs/python/tf/contrib/seq2seq/dynamic_decode)

In [11]:
def decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                         end_of_sequence_id, max_target_sequence_length,
                         vocab_size, output_layer, batch_size, keep_prob):
    """
    Create a decoding layer for inference
    :param encoder_state: Encoder state
    :param dec_cell: Decoder RNN Cell
    :param dec_embeddings: Decoder embeddings
    :param start_of_sequence_id: GO ID
    :param end_of_sequence_id: EOS Id
    :param max_target_sequence_length: Maximum length of target sequences
    :param vocab_size: Size of decoder/target vocabulary
    :param decoding_scope: TenorFlow Variable Scope for decoding
    :param output_layer: Function to apply the output layer
    :param batch_size: Batch size
    :param keep_prob: Dropout keep probability
    :return: BasicDecoderOutput containing inference logits and sample_id
    """
    # TODO: Implement Function
    # create start_token tensor (64, )

    start_token = tf.tile(tf.constant([start_of_sequence_id], dtype=tf.int32), [batch_size], name='start_token')
    
    inf_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings,
                                                          start_token,
                                                          end_of_sequence_id)
    
    inf_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, inf_helper, encoder_state, output_layer)
    
    basic_decoder_output = tf.contrib.seq2seq.dynamic_decode(inf_decoder,
                                                            impute_finished=True,
                                                            maximum_iterations=max_target_sequence_length)[0]
    
    
    #object, rnn output tensor shape (64, ?, 1000) --> float 32, sample ID shape (64, ?) --> int32)
    return basic_decoder_output 



"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer_infer(decoding_layer_infer)

Tests Passed


### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

* Embed the target sequences
* Construct the decoder LSTM cell (just like you constructed the encoder cell above)
* Create an output layer to map the outputs of the decoder to the elements of our vocabulary
* Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, target_sequence_length, max_target_sequence_length, output_layer, keep_prob)` function to get the training logits.
* Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, max_target_sequence_length, vocab_size, output_layer, batch_size, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.

In [19]:
def decoding_layer(dec_input, encoder_state,
                   target_sequence_length, max_target_sequence_length,
                   rnn_size,
                   num_layers, target_vocab_to_int, target_vocab_size,
                   batch_size, keep_prob, decoding_embedding_size):
    """
    Create decoding layer
    :param dec_input: Decoder input --> Tensor("Placeholder:0", shape=(64, 22), dtype=int32)
    :param encoder_state: Encoder state --> LSTMStateTuple(c=<tf.Tensor 'Placeholder_4:0' shape=(?, 512) dtype=float32>
    :param target_sequence_length: The lengths of each sequence in the target batch
    :param max_target_sequence_length: Maximum length of target sequences
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :param target_vocab_size: Size of target vocabulary --> 1000
    :param batch_size: The size of the batch
    :param keep_prob: Dropout keep probability
    :param decoding_embedding_size: Decoding embedding size --> 200
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    start_of_sequence_id = target_vocab_to_int['<GO>']
    end_of_sequence_id = target_vocab_to_int['<EOS>']

    # embed target seq
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(dec_embeddings, dec_input)

    # make LSTM decoder cell
    def make_cell(rnn_size):
        return tf.contrib.rnn.LSTMCell(rnn_size)

    dec_cell = tf.contrib.rnn.DropoutWrapper(
                                tf.contrib.rnn.MultiRNNCell(
                                [make_cell(rnn_size) for _ in range(num_layers)]),
                                input_keep_prob=keep_prob)
    
    # create output layer
    output_layer = Dense(target_vocab_size,
                        kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
        
    with tf.variable_scope('decode'):
        # use functions created above to get training and inference logits
        train_basic_decoder_output = decoding_layer_train(encoder_state, dec_cell, dec_embed_input, 
                                            target_sequence_length, max_target_sequence_length,
                                            output_layer, keep_prob)

    with tf.variable_scope('decode', reuse=True):
        inf_basic_decoder_output = decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id,
                                            end_of_sequence_id, max_target_sequence_length, target_vocab_size,
                                            output_layer, batch_size, keep_prob)


    return (train_basic_decoder_output, inf_basic_decoder_output)
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_decoding_layer(decoding_layer)

Tests Passed


In [33]:
def seq2seq_model(input_data, target_data, keep_prob, batch_size,
                  source_sequence_length, target_sequence_length,
                  max_target_sentence_length,
                  source_vocab_size, target_vocab_size,
                  enc_embedding_size, dec_embedding_size,
                  rnn_size, num_layers, target_vocab_to_int):
    """
    Build the Sequence-to-Sequence part of the neural network
    :param input_data: Input placeholder
    :param target_data: Target placeholder
    :param keep_prob: Dropout keep probability placeholder
    :param batch_size: Batch Size
    :param source_sequence_length: Sequence Lengths of source sequences in the batch
    :param target_sequence_length: Sequence Lengths of target sequences in the batch
    :param source_vocab_size: Source vocabulary size
    :param target_vocab_size: Target vocabulary size
    :param enc_embedding_size: Decoder embedding size
    :param dec_embedding_size: Encoder embedding size
    :param rnn_size: RNN Size
    :param num_layers: Number of layers
    :param target_vocab_to_int: Dictionary to go from the target words to an id
    :return: Tuple of (Training BasicDecoderOutput, Inference BasicDecoderOutput)
    """
    # TODO: Implement Function
    
    # encoding the input --> returns a tensor and LSTM state; using only state
    _, enc_state = encoding_layer(input_data, rnn_size, num_layers, keep_prob,
                         source_sequence_length, source_vocab_size,
                         enc_embedding_size)
    
    # prep target sequences to decoder in training
    dec = process_decoder_input(target_data, target_vocab_to_int, batch_size)
    
    # send encoder state and decoder sequence to decoders
    training_basic_decoder_output, inference_basic_decoder_output = decoding_layer(
                            target_data, enc_state, target_sequence_length,
                            max_target_sentence_length, rnn_size, num_layers,
                            target_vocab_to_int, target_vocab_size, batch_size,
                            keep_prob, dec_embedding_size)
    
    return (training_basic_decoder_output, inference_basic_decoder_output)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_seq2seq_model(seq2seq_model)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `num_layers` to the number of layers.
- Set `encoding_embedding_size` to the size of the embedding for the encoder.
- Set `decoding_embedding_size` to the size of the embedding for the decoder.
- Set `learning_rate` to the learning rate.
- Set `keep_probability` to the Dropout keep probability
- Set `display_step` to state how many steps between each debug output statement

In [72]:
# Number of Epochs
epochs = 3
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 50
# Number of Layers
num_layers = 2
# Embedding Size
encoding_embedding_size = 15
decoding_embedding_size = 15
# Learning Rate
learning_rate = 0.1
# Dropout Keep Probability
keep_probability = 0.65
display_step = 1

### Build the Graph
Build the graph using the neural network you implemented.

In [73]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
save_path = 'checkpoints/dev'
(source_int_text, target_int_text), (source_vocab_to_int, target_vocab_to_int), _ = helper.load_preprocess()
max_target_sentence_length = max([len(sentence) for sentence in source_int_text])

train_graph = tf.Graph()
with train_graph.as_default():
    input_data, targets, lr, keep_prob, target_sequence_length, max_target_sequence_length, source_sequence_length = model_inputs()

    #sequence_length = tf.placeholder_with_default(max_target_sentence_length, None, name='sequence_length')
    input_shape = tf.shape(input_data)

    train_logits, inference_logits = seq2seq_model(tf.reverse(input_data, [-1]),
                                                   targets,
                                                   keep_prob,
                                                   batch_size,
                                                   source_sequence_length,
                                                   target_sequence_length,
                                                   max_target_sequence_length,
                                                   len(source_vocab_to_int),
                                                   len(target_vocab_to_int),
                                                   encoding_embedding_size,
                                                   decoding_embedding_size,
                                                   rnn_size,
                                                   num_layers,
                                                   target_vocab_to_int)


    training_logits = tf.identity(train_logits.rnn_output, name='logits')
    inference_logits = tf.identity(inference_logits.sample_id, name='predictions')

    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(
            training_logits,
            targets,
            masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)


Batch and pad the source and target sequences

In [74]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def pad_sentence_batch(sentence_batch, pad_int):
    """Pad sentences with <PAD> so that each sentence of a batch has the same length"""
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [pad_int] * (max_sentence - len(sentence)) for sentence in sentence_batch]


def get_batches(sources, targets, batch_size, source_pad_int, target_pad_int):
    """Batch targets, sources, and the lengths of their sentences together"""
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size

        # Slice the right amount for the batch
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]

        # Pad
        pad_sources_batch = np.array(pad_sentence_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_sentence_batch(targets_batch, target_pad_int))

        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))

        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))

        yield pad_sources_batch, pad_targets_batch, pad_source_lengths, pad_targets_lengths


### Train
Train the neural network on the preprocessed data. If you have a hard time getting a good loss, check the forms to see if anyone is having the same problem.

In [75]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
def get_accuracy(target, logits):
    """
    Calculate accuracy
    """
    max_seq = max(target.shape[1], logits.shape[1])
    if max_seq - target.shape[1]:
        target = np.pad(
            target,
            [(0,0),(0,max_seq - target.shape[1])],
            'constant')
    if max_seq - logits.shape[1]:
        logits = np.pad(
            logits,
            [(0,0),(0,max_seq - logits.shape[1])],
            'constant')

    return np.mean(np.equal(target, logits))

# Split data to training and validation sets
train_source = source_int_text[batch_size:]
train_target = target_int_text[batch_size:]
valid_source = source_int_text[:batch_size]
valid_target = target_int_text[:batch_size]
(valid_sources_batch, valid_targets_batch, valid_sources_lengths, valid_targets_lengths ) = next(get_batches(valid_source,
                                                                                                             valid_target,
                                                                                                             batch_size,
                                                                                                             source_vocab_to_int['<PAD>'],
                                                                                                             target_vocab_to_int['<PAD>']))                                                                                                  
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(epochs):
        for batch_i, (source_batch, target_batch, sources_lengths, targets_lengths) in enumerate(
                get_batches(train_source, train_target, batch_size,
                            source_vocab_to_int['<PAD>'],
                            target_vocab_to_int['<PAD>'])):

            _, loss = sess.run(
                [train_op, cost],
                {input_data: source_batch,
                 targets: target_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths,
                 keep_prob: keep_probability})


            if batch_i % display_step == 0 and batch_i > 0:


                batch_train_logits = sess.run(
                    inference_logits,
                    {input_data: source_batch,
                     source_sequence_length: sources_lengths,
                     target_sequence_length: targets_lengths,
                     keep_prob: 1.0})


                batch_valid_logits = sess.run(
                    inference_logits,
                    {input_data: valid_sources_batch,
                     source_sequence_length: valid_sources_lengths,
                     target_sequence_length: valid_targets_lengths,
                     keep_prob: 1.0})

                train_acc = get_accuracy(target_batch, batch_train_logits)

                valid_acc = get_accuracy(valid_targets_batch, batch_valid_logits)

                print('Epoch {:>3} Batch {:>4}/{} - Train Accuracy: {:>6.4f}, Validation Accuracy: {:>6.4f}, Loss: {:>6.4f}'
                      .format(epoch_i, batch_i, len(source_int_text) // batch_size, train_acc, valid_acc, loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_path)
    print('Model Trained and Saved')

Epoch   0 Batch    1/1077 - Train Accuracy: 0.2324, Validation Accuracy: 0.3189, Loss: 4.1578
Epoch   0 Batch    2/1077 - Train Accuracy: 0.2064, Validation Accuracy: 0.3050, Loss: 4.1322
Epoch   0 Batch    3/1077 - Train Accuracy: 0.2500, Validation Accuracy: 0.3185, Loss: 4.9053
Epoch   0 Batch    4/1077 - Train Accuracy: 0.2160, Validation Accuracy: 0.2898, Loss: 4.2857
Epoch   0 Batch    5/1077 - Train Accuracy: 0.2676, Validation Accuracy: 0.3189, Loss: 4.1209
Epoch   0 Batch    6/1077 - Train Accuracy: 0.2586, Validation Accuracy: 0.3168, Loss: 3.8229
Epoch   0 Batch    7/1077 - Train Accuracy: 0.2465, Validation Accuracy: 0.3278, Loss: 3.8140
Epoch   0 Batch    8/1077 - Train Accuracy: 0.2672, Validation Accuracy: 0.3363, Loss: 3.6995
Epoch   0 Batch    9/1077 - Train Accuracy: 0.2770, Validation Accuracy: 0.3409, Loss: 3.6373
Epoch   0 Batch   10/1077 - Train Accuracy: 0.2401, Validation Accuracy: 0.3441, Loss: 3.8009
Epoch   0 Batch   11/1077 - Train Accuracy: 0.1257, Validati

Epoch   0 Batch   89/1077 - Train Accuracy: 0.1066, Validation Accuracy: 0.0898, Loss: 0.6386
Epoch   0 Batch   90/1077 - Train Accuracy: 0.0891, Validation Accuracy: 0.0863, Loss: 0.6708
Epoch   0 Batch   91/1077 - Train Accuracy: 0.0670, Validation Accuracy: 0.0835, Loss: 0.5177
Epoch   0 Batch   92/1077 - Train Accuracy: 0.0800, Validation Accuracy: 0.0891, Loss: 0.6336
Epoch   0 Batch   93/1077 - Train Accuracy: 0.0828, Validation Accuracy: 0.0863, Loss: 0.5829
Epoch   0 Batch   94/1077 - Train Accuracy: 0.0781, Validation Accuracy: 0.0859, Loss: 0.5396
Epoch   0 Batch   95/1077 - Train Accuracy: 0.0822, Validation Accuracy: 0.0835, Loss: 0.5909
Epoch   0 Batch   96/1077 - Train Accuracy: 0.0750, Validation Accuracy: 0.0795, Loss: 0.5252
Epoch   0 Batch   97/1077 - Train Accuracy: 0.0680, Validation Accuracy: 0.0813, Loss: 0.5530
Epoch   0 Batch   98/1077 - Train Accuracy: 0.0837, Validation Accuracy: 0.0913, Loss: 0.5117
Epoch   0 Batch   99/1077 - Train Accuracy: 0.1090, Validati

Epoch   0 Batch  177/1077 - Train Accuracy: 0.0979, Validation Accuracy: 0.0856, Loss: 0.2388
Epoch   0 Batch  178/1077 - Train Accuracy: 0.0883, Validation Accuracy: 0.0856, Loss: 0.1898
Epoch   0 Batch  179/1077 - Train Accuracy: 0.1123, Validation Accuracy: 0.0941, Loss: 0.1932
Epoch   0 Batch  180/1077 - Train Accuracy: 0.1012, Validation Accuracy: 0.0962, Loss: 0.1559
Epoch   0 Batch  181/1077 - Train Accuracy: 0.0988, Validation Accuracy: 0.0962, Loss: 0.2128
Epoch   0 Batch  182/1077 - Train Accuracy: 0.0956, Validation Accuracy: 0.0955, Loss: 0.1498
Epoch   0 Batch  183/1077 - Train Accuracy: 0.1027, Validation Accuracy: 0.0938, Loss: 0.1748
Epoch   0 Batch  184/1077 - Train Accuracy: 0.0980, Validation Accuracy: 0.0852, Loss: 0.1365
Epoch   0 Batch  185/1077 - Train Accuracy: 0.1094, Validation Accuracy: 0.0781, Loss: 0.1525
Epoch   0 Batch  186/1077 - Train Accuracy: 0.0794, Validation Accuracy: 0.0838, Loss: 0.2000
Epoch   0 Batch  187/1077 - Train Accuracy: 0.0973, Validati

Epoch   0 Batch  265/1077 - Train Accuracy: 0.0809, Validation Accuracy: 0.0639, Loss: 0.0808
Epoch   0 Batch  266/1077 - Train Accuracy: 0.0789, Validation Accuracy: 0.0746, Loss: 0.0846
Epoch   0 Batch  267/1077 - Train Accuracy: 0.0874, Validation Accuracy: 0.0810, Loss: 0.0991
Epoch   0 Batch  268/1077 - Train Accuracy: 0.0828, Validation Accuracy: 0.0806, Loss: 0.0949
Epoch   0 Batch  269/1077 - Train Accuracy: 0.1032, Validation Accuracy: 0.0888, Loss: 0.1293
Epoch   0 Batch  270/1077 - Train Accuracy: 0.0984, Validation Accuracy: 0.0866, Loss: 0.1065
Epoch   0 Batch  271/1077 - Train Accuracy: 0.0984, Validation Accuracy: 0.0877, Loss: 0.0925
Epoch   0 Batch  272/1077 - Train Accuracy: 0.0882, Validation Accuracy: 0.0856, Loss: 0.1263
Epoch   0 Batch  273/1077 - Train Accuracy: 0.0837, Validation Accuracy: 0.0835, Loss: 0.0929
Epoch   0 Batch  274/1077 - Train Accuracy: 0.0871, Validation Accuracy: 0.0771, Loss: 0.0887
Epoch   0 Batch  275/1077 - Train Accuracy: 0.0848, Validati

Epoch   0 Batch  353/1077 - Train Accuracy: 0.0724, Validation Accuracy: 0.0572, Loss: 0.0757
Epoch   0 Batch  354/1077 - Train Accuracy: 0.0758, Validation Accuracy: 0.0604, Loss: 0.0859
Epoch   0 Batch  355/1077 - Train Accuracy: 0.0789, Validation Accuracy: 0.0728, Loss: 0.0844
Epoch   0 Batch  356/1077 - Train Accuracy: 0.0941, Validation Accuracy: 0.0824, Loss: 0.0893
Epoch   0 Batch  357/1077 - Train Accuracy: 0.1060, Validation Accuracy: 0.0952, Loss: 0.0835
Epoch   0 Batch  358/1077 - Train Accuracy: 0.0826, Validation Accuracy: 0.0749, Loss: 0.0807
Epoch   0 Batch  359/1077 - Train Accuracy: 0.0668, Validation Accuracy: 0.0604, Loss: 0.0715
Epoch   0 Batch  360/1077 - Train Accuracy: 0.0875, Validation Accuracy: 0.0938, Loss: 0.0877
Epoch   0 Batch  361/1077 - Train Accuracy: 0.0683, Validation Accuracy: 0.0700, Loss: 0.0839
Epoch   0 Batch  362/1077 - Train Accuracy: 0.0748, Validation Accuracy: 0.0607, Loss: 0.0965
Epoch   0 Batch  363/1077 - Train Accuracy: 0.0770, Validati

...

Epoch   2 Batch  669/1077 - Train Accuracy: 0.0629, Validation Accuracy: 0.0689, Loss: 0.1909
Epoch   2 Batch  670/1077 - Train Accuracy: 0.0774, Validation Accuracy: 0.0604, Loss: 0.2993
Epoch   2 Batch  671/1077 - Train Accuracy: 0.0502, Validation Accuracy: 0.0494, Loss: 0.2529
Epoch   2 Batch  672/1077 - Train Accuracy: 0.0506, Validation Accuracy: 0.0554, Loss: 0.2483
Epoch   2 Batch  673/1077 - Train Accuracy: 0.0677, Validation Accuracy: 0.0863, Loss: 0.2482
Epoch   2 Batch  674/1077 - Train Accuracy: 0.0684, Validation Accuracy: 0.0682, Loss: 0.2656
Epoch   2 Batch  675/1077 - Train Accuracy: 0.0714, Validation Accuracy: 0.0671, Loss: 0.2373
Epoch   2 Batch  676/1077 - Train Accuracy: 0.0794, Validation Accuracy: 0.0732, Loss: 0.2276
Epoch   2 Batch  677/1077 - Train Accuracy: 0.0711, Validation Accuracy: 0.0785, Loss: 0.3285
Epoch   2 Batch  678/1077 - Train Accuracy: 0.0517, Validation Accuracy: 0.0540, Loss: 0.2169
Epoch   2 Batch  679/1077 - Train Accuracy: 0.0551, Validati

Epoch   2 Batch  757/1077 - Train Accuracy: 0.0210, Validation Accuracy: 0.0128, Loss: 0.2417
Epoch   2 Batch  758/1077 - Train Accuracy: 0.0264, Validation Accuracy: 0.0124, Loss: 0.2030
Epoch   2 Batch  759/1077 - Train Accuracy: 0.0164, Validation Accuracy: 0.0142, Loss: 0.2310
Epoch   2 Batch  760/1077 - Train Accuracy: 0.0273, Validation Accuracy: 0.0128, Loss: 0.2421
Epoch   2 Batch  761/1077 - Train Accuracy: 0.0206, Validation Accuracy: 0.0149, Loss: 0.2611
Epoch   2 Batch  762/1077 - Train Accuracy: 0.0332, Validation Accuracy: 0.0352, Loss: 0.2855
Epoch   2 Batch  763/1077 - Train Accuracy: 0.0268, Validation Accuracy: 0.0281, Loss: 0.2588
Epoch   2 Batch  764/1077 - Train Accuracy: 0.0288, Validation Accuracy: 0.0281, Loss: 0.2586
Epoch   2 Batch  765/1077 - Train Accuracy: 0.0191, Validation Accuracy: 0.0210, Loss: 0.2372
Epoch   2 Batch  766/1077 - Train Accuracy: 0.0164, Validation Accuracy: 0.0199, Loss: 0.2498
Epoch   2 Batch  767/1077 - Train Accuracy: 0.0199, Validati

Epoch   2 Batch  845/1077 - Train Accuracy: 0.0844, Validation Accuracy: 0.1058, Loss: 0.1882
Epoch   2 Batch  846/1077 - Train Accuracy: 0.0887, Validation Accuracy: 0.1051, Loss: 0.2500
Epoch   2 Batch  847/1077 - Train Accuracy: 0.0840, Validation Accuracy: 0.0866, Loss: 0.2682
Epoch   2 Batch  848/1077 - Train Accuracy: 0.0598, Validation Accuracy: 0.0469, Loss: 0.1914
Epoch   2 Batch  849/1077 - Train Accuracy: 0.0512, Validation Accuracy: 0.0430, Loss: 0.1812
Epoch   2 Batch  850/1077 - Train Accuracy: 0.0476, Validation Accuracy: 0.0398, Loss: 0.2215
Epoch   2 Batch  851/1077 - Train Accuracy: 0.0406, Validation Accuracy: 0.0369, Loss: 0.2366
Epoch   2 Batch  852/1077 - Train Accuracy: 0.0418, Validation Accuracy: 0.0327, Loss: 0.2429
Epoch   2 Batch  853/1077 - Train Accuracy: 0.0418, Validation Accuracy: 0.0352, Loss: 0.1472
Epoch   2 Batch  854/1077 - Train Accuracy: 0.0508, Validation Accuracy: 0.0398, Loss: 0.2550
Epoch   2 Batch  855/1077 - Train Accuracy: 0.0414, Validati

Epoch   2 Batch  933/1077 - Train Accuracy: 0.0531, Validation Accuracy: 0.0451, Loss: 0.2008
Epoch   2 Batch  934/1077 - Train Accuracy: 0.0496, Validation Accuracy: 0.0487, Loss: 0.2006
Epoch   2 Batch  935/1077 - Train Accuracy: 0.0477, Validation Accuracy: 0.0472, Loss: 0.1832
Epoch   2 Batch  936/1077 - Train Accuracy: 0.0577, Validation Accuracy: 0.0426, Loss: 0.1786
Epoch   2 Batch  937/1077 - Train Accuracy: 0.0539, Validation Accuracy: 0.0501, Loss: 0.2064
Epoch   2 Batch  938/1077 - Train Accuracy: 0.0664, Validation Accuracy: 0.0476, Loss: 0.2527
Epoch   2 Batch  939/1077 - Train Accuracy: 0.0457, Validation Accuracy: 0.0479, Loss: 0.1970
Epoch   2 Batch  940/1077 - Train Accuracy: 0.0488, Validation Accuracy: 0.0522, Loss: 0.2006
Epoch   2 Batch  941/1077 - Train Accuracy: 0.0640, Validation Accuracy: 0.0437, Loss: 0.1948
Epoch   2 Batch  942/1077 - Train Accuracy: 0.0570, Validation Accuracy: 0.0572, Loss: 0.2067
Epoch   2 Batch  943/1077 - Train Accuracy: 0.0461, Validati

Epoch   2 Batch 1021/1077 - Train Accuracy: 0.0335, Validation Accuracy: 0.0288, Loss: 0.1837
Epoch   2 Batch 1022/1077 - Train Accuracy: 0.0279, Validation Accuracy: 0.0259, Loss: 0.1513
Epoch   2 Batch 1023/1077 - Train Accuracy: 0.0220, Validation Accuracy: 0.0156, Loss: 0.1879
Epoch   2 Batch 1024/1077 - Train Accuracy: 0.0234, Validation Accuracy: 0.0149, Loss: 0.2146
Epoch   2 Batch 1025/1077 - Train Accuracy: 0.0186, Validation Accuracy: 0.0188, Loss: 0.2103
Epoch   2 Batch 1026/1077 - Train Accuracy: 0.0231, Validation Accuracy: 0.0110, Loss: 0.2162
Epoch   2 Batch 1027/1077 - Train Accuracy: 0.0227, Validation Accuracy: 0.0117, Loss: 0.1660
Epoch   2 Batch 1028/1077 - Train Accuracy: 0.0171, Validation Accuracy: 0.0121, Loss: 0.1508
Epoch   2 Batch 1029/1077 - Train Accuracy: 0.0195, Validation Accuracy: 0.0128, Loss: 0.1862
Epoch   2 Batch 1030/1077 - Train Accuracy: 0.0289, Validation Accuracy: 0.0131, Loss: 0.2064
Epoch   2 Batch 1031/1077 - Train Accuracy: 0.0206, Validati

### Save Parameters
Save the `batch_size` and `save_path` parameters for inference.

In [76]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params(save_path)

# Checkpoint

In [77]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = helper.load_preprocess()
load_path = helper.load_params()

## Sentence to Sequence
To feed a sentence into the model for translation, you first need to preprocess it.  Implement the function `sentence_to_seq()` to preprocess new sentences.

- Convert the sentence to lowercase
- Convert words into ids using `vocab_to_int`
 - Convert words not in the vocabulary, to the `<UNK>` word id.

In [78]:
def sentence_to_seq(sentence, vocab_to_int):
    """
    Convert a sentence to a sequence of ids
    :param sentence: String
    :param vocab_to_int: Dictionary to go from the words to an id
    :return: List of word ids
    """
    # TODO: Implement Function
    
    word_tokens = sentence.lower().split()
    unk_value = vocab_to_int['<UNK>']
    word_ids = [vocab_to_int.get(word, unk_value) for word in word_tokens]
    
    return word_ids


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_sentence_to_seq(sentence_to_seq)

Tests Passed


## Translate
This will translate `translate_sentence` from English to French.

In [79]:
translate_sentence = 'he saw a green truck.'


"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
translate_sentence = sentence_to_seq(translate_sentence, source_vocab_to_int)

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_path + '.meta')
    loader.restore(sess, load_path)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')

    translate_logits = sess.run(logits, {input_data: [translate_sentence]*batch_size,
                                         target_sequence_length: [len(translate_sentence)*2]*batch_size,
                                         source_sequence_length: [len(translate_sentence)]*batch_size,
                                         keep_prob: 1.0})[0]

print('Input')
print('  Word Ids:      {}'.format([i for i in translate_sentence]))
print('  English Words: {}'.format([source_int_to_vocab[i] for i in translate_sentence]))

print('\nPrediction')
print('  Word Ids:      {}'.format([i for i in translate_logits]))
print('  French Words: {}'.format(" ".join([target_int_to_vocab[i] for i in translate_logits])))


INFO:tensorflow:Restoring parameters from checkpoints/dev
Input
  Word Ids:      [59, 85, 100, 83, 2]
  English Words: ['he', 'saw', 'a', 'green', '<UNK>']

Prediction
  Word Ids:      [324, 115, 352, 352, 167, 167, 167, 167, 167, 167]
  French Words: j'aime pêche la la . . . . . .


## Imperfect Translation
You might notice that some sentences translate better than others.  Since the dataset you're using only has a vocabulary of 227 English words of the thousands that you use, you're only going to see good results using these words.  For this project, you don't need a perfect translation. However, if you want to create a better translation model, you'll need better data.

You can train on the [WMT10 French-English corpus](http://www.statmt.org/wmt10/training-giga-fren.tar).  This dataset has more vocabulary and richer in topics discussed.  However, this will take you days to train, so make sure you've a GPU and the neural network is performing well on dataset we provided.  Just make sure you play with the WMT10 corpus after you've submitted this project.
## Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_language_translation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.